In [1]:
import getpass
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas
from astropy.time import Time

import lsst.geom
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler, DatasetType, CollectionType
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [2]:
my_collection_identifier = 'custom_coadd_window1_cl00_spe'
uname = os.environ['USER']
my_outputCollection = 'u/' + uname + '/'+my_collection_identifier

print(my_outputCollection)

u/ameisner/custom_coadd_window1_cl00_spe


In [3]:
my_inputCollection = 'u/' + uname + '/custom_coadd_window1_cl00'
print(my_inputCollection)

u/ameisner/custom_coadd_window1_cl00


In [4]:
simpleButler = SimplePipelineExecutor.prep_butler('dp02', 
                                                  inputs=[my_inputCollection], 
                                                  output=my_outputCollection)

In [5]:
simpleButler.registry.getCollectionChain(my_outputCollection)

CollectionSearch(('u/ameisner/custom_coadd_window1_cl00_spe/20231106T020023Z', 'u/ameisner/custom_coadd_window1_cl00'))

In [6]:
yaml_file = '$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml'
steps = 'detection,mergeDetections,deblend,measure'
my_uri = yaml_file + '#' + steps
print(my_uri)

$DRP_PIPE_DIR/pipelines/LSSTCam-imSim/DRP-test-med-1.yaml#detection,mergeDetections,deblend,measure


In [7]:
sourceDetectionPipeline = Pipeline.from_uri(my_uri)

In [8]:
# -c detection:detection.thresholdValue=10 \
# -c detection:detection.thresholdType="stdev" \
# -c deblend:multibandDeblend.maxIter=20 \
# -c measure:doPropagateFlags=False \


sourceDetectionPipeline.addConfigOverride('detection', 'detection.thresholdValue', 10.0)
sourceDetectionPipeline.addConfigOverride('detection', 'detection.thresholdType', "stdev")
sourceDetectionPipeline.addConfigOverride('deblend', 'multibandDeblend.maxIter', 20)
sourceDetectionPipeline.addConfigOverride('measure', 'doPropagateFlags', False)

In [10]:
#note that this gives a warning that can apparently be ignored

from lsst.ctrl.mpexec import pipeline2dot
pipeline2dot(sourceDetectionPipeline, '/home/' + uname + '/custom_coadd_qgraph.dot')

! dot -Tpng /home/$USER/custom_coadd_qgraph.dot > /home/$USER/custom_coadd_qgraph.png

In [13]:
queryString = f"tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'"

In [14]:
print(queryString)

tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'


In [15]:
spe = SimplePipelineExecutor.from_pipeline(sourceDetectionPipeline, 
                                           where=queryString, 
                                           butler=simpleButler)

In [16]:
quanta = spe.run()

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/models.py:119: RuntimeWarning: invalid value encountered in true_divide
  if np.any(edge_flux/edge_mask > self.bg_thresh*self.bg_rms[:, None, None]):
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/measure.py:35: RuntimeWarning: invalid value encountered in multiply
  denominator = (psfs * noise) * psfs
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/parameters.py:302: RuntimeWarning: invalid value encountered in true_divide
  _z = self.prox(z - gamma/step * psi * (z-self.x), gamma)
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/parameters.py:302: RuntimeWarning: invalid value encountered in subtract
  _z = self.prox(z - gamma/step * psi * (z-self

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/measure.py:85: RuntimeWarning: divide by zero encountered in true_divide
  ratio = numerator / denominator
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/scarlet/gd32b658ba2+4083830bf8/lib/python/scarlet/lite/measure.py:85: RuntimeWarning: invalid value encountered in true_divide
  ratio = numerator / denominator
